In [1]:
# packages to store and manipulate data
import pandas as pd
import numpy as np

# plotting packages
import matplotlib.pyplot as plt
# model building package
import sklearn
import os
# package to clean text
import re
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch-covid', 'port': 9200}])

Elasticsearch tiene varios tipos de query en la que se calcula el score de diferentes maneras, hay que buscar la que mejor se ajusta al problema.

In [2]:
def serch_query_string(query,
                         index_pattern='covid*',
                         size=10,
                         fragments=0,
                         text_field='article_text'):
    return es.search(
        index=index_pattern,
        body={
            "from": 0, "size": size,         
            "query": {
                    "query_string" : {
                        "query" : query,
                        "default_field" : text_field,
                        "minimum_should_match": 1
                    }
                },
             "highlight": {
                "fields": {
                    "*": {}
                },
                "number_of_fragments": fragments,
                 "fragment_size": 500,
                 "pre_tags":["<strong><u>"],
                 "post_tags":["</u></strong>"]
            }
        })['hits']['hits']

In [3]:
from IPython.core.display import display, HTML
def get(fragments,text):
    for result in serch_query_string(text,fragments=fragments):
        print('Score: {}'.format(result['_score']))
        for fragment in result['highlight']['article_text']:
            display(HTML(fragment))
            print('__')
        print()

Además del score puede indicar donde hizo el match. Puede develver cierto número de fragmentos que contienen el match o el documento completo

In [4]:
get(3,'¿Cuánto demora el proceso de registro del test desarrollado por Udelar y el Institut Pasteur?')

Score: 19.320383


__


__


__

Score: 3.4270847


__


...

__

Score: 1.2444594


__

Score: 1.1844817


__

Score: 1.0313272


__


__



In [5]:
get(0,'¿Cuánto demora el proceso de registro del test desarrollado por Udelar y el Institut Pasteur?')

Score: 19.320383


__

Score: 3.4270847


__

Score: 3.3902993


__

Score: 2.6989446


__

Score: 2.2033873


...

__

Score: 1.7947996


__

Score: 1.2444594


__

Score: 1.1844817


__

Score: 1.0313272


__



In [6]:
get(0,'¿Cuántos días habrían pasado desde que se iniciaron las medidas de aislamiento si se levantan después de Turismo?')

Score: 7.843624


__

Score: 6.102944


__

Score: 4.1314917


__

Score: 3.8811946


__

Score: 3.2873454


...

__

Score: 2.875367


__

Score: 2.4118094


__

Score: 1.6667751


__

Score: 1.4902495


__

